In [1]:
DROP_OUT = 0.1

In [2]:
# Ensure necessary libraries are installed
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torchsummary import summary
except ModuleNotFoundError:
    raise ModuleNotFoundError("The 'torch' package is not installed. Please install it using 'pip install torch'.")
DROP_OUT = 0.1
class CIFAR10Net(nn.Module):
    def __init__(self, num_classes=10):
        super(CIFAR10Net, self).__init__()

        # Block 1 (Receptive Field: 3 -> 7 with dilation)
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, dilation=1, padding=1, groups=1),  # RF: 3
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(DROP_OUT),
            nn.Conv2d(32, 32, kernel_size=3, dilation=2, padding=2, groups=32),  # RF: 7 (Depthwise)
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(DROP_OUT),
            nn.Conv2d(32, 32, kernel_size=1),  # Pointwise convolution (RF unchanged: 7)
            nn.BatchNorm2d(32),
            nn.ReLU()
        )

        # Block 2 (Receptive Field: 7 -> 15 -> 23)
        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, dilation=2, padding=2, groups=32),  # RF: 15 (Depthwise)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(DROP_OUT),
            nn.Conv2d(64, 64, kernel_size=3, dilation=3, padding=3, groups=64),  # RF: 23 (Depthwise)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(DROP_OUT),
            nn.Conv2d(64, 64, kernel_size=1),  # Pointwise convolution (RF unchanged: 23)
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        # Block 3 (Receptive Field: 23 -> 39 -> 55)
        self.block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, dilation=3, padding=3, groups=64),  # RF: 39 (Depthwise)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(DROP_OUT),
            nn.Conv2d(128, 128, kernel_size=3, dilation=4, padding=4, groups=128),  # RF: 55 (Depthwise)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(DROP_OUT),
            nn.Conv2d(128, 128, kernel_size=1),  # Pointwise convolution (RF unchanged: 55)
            nn.BatchNorm2d(128),
            nn.ReLU()
        )

        # Block 4 (Receptive Field: 55 -> 87 -> 119)
        self.block4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, dilation=4, padding=4, groups=128),  # RF: 87 (Depthwise)
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(DROP_OUT),
            nn.Conv2d(256, 256, kernel_size=3, dilation=5, padding=5, groups=256),  # RF: 119 (Depthwise)
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(DROP_OUT),
            nn.Conv2d(256, 256, kernel_size=1),  # Pointwise convolution (RF unchanged: 119)
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)  # Global pooling (RF covers entire input)
        )

        # Fully connected layers
        self.fc1 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(DROP_OUT)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x




In [3]:
from torchvision import datasets
import numpy as np
from PIL import Image

class CIFAR10Dataset:
    def __init__(self, root="./data", train=True, transform=None):
        self.dataset = datasets.CIFAR10(root=root, train=train, download=True)
        self.transform = transform

        # Calculate mean and std
        if train:
            data = np.array(self.dataset.data, dtype=np.float32)
            self.mean = data.mean(axis=(0,1,2))/255.0
            self.std = data.std(axis=(0,1,2))/255.0

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, label = self.dataset[idx]

        # Convert PIL Image to numpy array
        if isinstance(img, Image.Image):
            img = np.array(img)

        if self.transform:
            img = self.transform(img, train=self.dataset.train)
        return img, label

In [4]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np

class Transforms:
    def __init__(self, means, stds):
        # Convert numpy arrays to lists if necessary
        if isinstance(means, np.ndarray):
            means = means.tolist()
        if isinstance(stds, np.ndarray):
            stds = stds.tolist()

        self.train_transform = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
            A.CoarseDropout(
                max_holes=1, max_height=16, max_width=16,
                min_holes=1, min_height=16, min_width=16,
                fill_value=[x * 255 for x in means],  # Convert to 0-255 range
                p=0.5
            ),
            A.Normalize(mean=means, std=stds),
            ToTensorV2()
        ])

        self.test_transform = A.Compose([
            A.Normalize(mean=means, std=stds),
            ToTensorV2()
        ])

    def __call__(self, img, train=True):
        # Convert PIL Image to numpy array if needed
        if not isinstance(img, np.ndarray):
            img = np.array(img)

        if train:
            return self.train_transform(image=img)["image"]
        return self.test_transform(image=img)["image"]

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchsummary import summary

def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\t'
                  f'Loss: {train_loss/(batch_idx+1):.6f} '
                  f'Accuracy: {100.*correct/total:.2f}%')

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    test_loss /= len(test_loader)
    accuracy = 100. * correct / total
    print(f'\nTest set: Average loss: {test_loss:.4f}, '
          f'Accuracy: {correct}/{total} ({accuracy:.2f}%)\n')
    return accuracy





In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Create datasets and dataloaders
train_dataset = CIFAR10Dataset(train=True)
transform = Transforms(means=train_dataset.mean, stds=train_dataset.std)

train_dataset.transform = transform
test_dataset = CIFAR10Dataset(train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

# Create model, optimizer and loss function
model = CIFAR10Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Print model summary
summary(model, (3, 32, 32))

cuda


100%|██████████| 170M/170M [00:03<00:00, 49.6MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]             320
       BatchNorm2d-6           [-1, 32, 32, 32]              64
              ReLU-7           [-1, 32, 32, 32]               0
           Dropout-8           [-1, 32, 32, 32]               0
            Conv2d-9           [-1, 32, 32, 32]           1,056
      BatchNorm2d-10           [-1, 32, 32, 32]              64
             ReLU-11           [-1, 32, 32, 32]               0
           Conv2d-12           [-1, 64, 32, 32]             640
      BatchNor

In [ ]:
# Training loop
best_acc = 0
for epoch in range(1, 100):
    train(model, device, train_loader, optimizer, criterion, epoch)
    accuracy = test(model, device, test_loader, criterion)

    if accuracy > best_acc:
        best_acc = accuracy
        torch.save(model.state_dict(), 'best_model.pth')

Epoch: 1 [0/50000 (0%)]	Loss: 1.629006 Accuracy: 41.41%
Epoch: 1 [12800/50000 (26%)]	Loss: 1.583827 Accuracy: 41.21%
Epoch: 1 [25600/50000 (51%)]	Loss: 1.536960 Accuracy: 42.88%
Epoch: 1 [38400/50000 (77%)]	Loss: 1.494583 Accuracy: 44.67%

Test set: Average loss: 1.5296, Accuracy: 4859/10000 (48.59%)

Epoch: 2 [0/50000 (0%)]	Loss: 1.356515 Accuracy: 50.78%
Epoch: 2 [12800/50000 (26%)]	Loss: 1.320445 Accuracy: 51.81%
Epoch: 2 [25600/50000 (51%)]	Loss: 1.304898 Accuracy: 52.41%
Epoch: 2 [38400/50000 (77%)]	Loss: 1.289786 Accuracy: 53.05%

Test set: Average loss: 1.1982, Accuracy: 5826/10000 (58.26%)

Epoch: 3 [0/50000 (0%)]	Loss: 1.471847 Accuracy: 42.19%
Epoch: 3 [12800/50000 (26%)]	Loss: 1.203111 Accuracy: 56.03%
Epoch: 3 [25600/50000 (51%)]	Loss: 1.191463 Accuracy: 56.53%
Epoch: 3 [38400/50000 (77%)]	Loss: 1.180886 Accuracy: 57.00%

Test set: Average loss: 1.1542, Accuracy: 6011/10000 (60.11%)

Epoch: 4 [0/50000 (0%)]	Loss: 1.082267 Accuracy: 63.28%
Epoch: 4 [12800/50000 (26%)]	Loss: 